# Intro

This is a replication of the 1995 paper by Blackburn and Neumark investigating bias in wage OLS estimations. The main delivery is that significant part of the bias is due to not taking into account the cognitive abilities. Once the estimates of cognitive abilities are takes as the controls in the regression, the strength of the education effect drops by around 40%.

### Data
NLSY 1979 is used for the research. To get the estimates of cognitive abilities, ASVAB test scores are regressed on the age so we avoid age bias, since the test is administered independent of the age of respondents. Then these estimates are used as controls. In this replication I use aggregated ASVAB residual score, which is different from the original research, that uses separate score residuals for each of ASVAB tests. Another difference is the year of data used.
To get aggregated experience I sum up the experience of all previous years for the respondents. Union coverage is derived by identifying whether the respondent had coverage from any of 13 employers.
Another controls are - ethnicity, SMSA residence, tenure, marital status, age, in some regressions - squared experience and age.
Since strong gender bias in the market is present, regressions are run separate for genders.

## Uploading the file
In this section I upload the necessary libraries and the files with NLSY79 data.

In [0]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [4]:
from google.colab import files

uploaded = files.upload()


for n, f in uploaded.items():
  with open(n,'wb') as file:
    file.write(f)

varMap = {'CPSHRP': 'wage',
          'FAM-1B': 'age',
          'Q15-1': 'ethnicity',
          'SMSARES': 'smsa',
          'MARSTAT-COL': 'marriage',
          'HGC': 'educ',
          'WKSWK-SLI': 'weeksWorked',
          'AFQT-1': 'testScore',
          'EMPLOYERS_ALL_UNION_1981.10': 'EMPLOYERS_ALL_UNION_1981.010',
          'EMPLOYERS_ALL_UNION_1981.11': 'EMPLOYERS_ALL_UNION_1981.011',
          'EMPLOYERS_ALL_UNION_1981.12': 'EMPLOYERS_ALL_UNION_1981.012',
          'EMPLOYERS_ALL_UNION_1981.13': 'EMPLOYERS_ALL_UNION_1981.013',
          'ASVAB-3': 'ASVABa',
          'ASVAB-4': 'ASVABb',
          'ASVAB-5': 'ASVABc',
          'ASVAB-6': 'ASVABd',
          'ASVAB-7': 'ASVABe',
          'ASVAB-8': 'ASVABf',
          'ASVAB-9': 'ASVABg',
          'ASVAB-10': 'ASVABh',}

## Data preprocessing

In this section I aggregate different union coverages to find if the respondent has any, do aggregation of the ASVAB scores, calculate tenure, total experience, find squared experience and squared age. Then I get rid of the variables I am not using directly in the regressions and get rid of the observations with missing values.

In [0]:
def unionCheck(x):
  check = 0
  for idx in range(1, 14):
    if x['EMPLOYERS_ALL_UNION_1981.0' + str(idx)] == 1:
      check = 1
  return check

def asvabAggregation(x):
  return sum([x['ASVAB-' + str(i)] if x['ASVAB-' + str(i)] > 0 else 0 for i in range(3, 11)])

def tenureCollapse(x):
  return sum([x['EMPLOYERS_ALL_TENURE_19' + str(i + 79) + '.01'] if x['EMPLOYERS_ALL_TENURE_19' + str(i + 79) + '.01'] > 0 else 0 for i in range(3)])

def totalExperience(x):
  return (x['WKSWK-SLI.1'] if x['WKSWK-SLI.1'] > 0 else 0) + (x['WKSWK-SLI.2'] if x['WKSWK-SLI.2'] > 0 else 0) + (x['weeksWorked'] if x['weeksWorked'] > 0 else 0)


In [6]:
df = pd.read_csv('oneMore.csv')
df.rename(columns = varMap, inplace = True)
df['unionCoverage'] = df.apply(unionCheck, axis = 1)
df['experience'] = df.apply(totalExperience, axis = 1)
df['experience'] = df.experience / float(max(df.experience))
df['tenure'] = df.apply(tenureCollapse, axis = 1)
# df['cogAbilities'] = df.apply(asvabAggregation, axis = 1)
df.drop(labels = [c for c in df.columns if c[:17] == 'EMPLOYERS_ALL_TEN'], axis = 1, inplace = True)
df.drop(labels = ['EMPLOYERS_ALL_UNION_1981.0' + str(idx) for idx in range(1, 14)], axis = 1, inplace = True)
df.drop(labels = ['WKSWK-SLI.1', 'WKSWK-SLI.2', 'weeksWorked'], axis = 1, inplace = True)
# df.drop(labels = ['ASVAB-' + str(i) for i in range(3, 11)], axis = 1, inplace = True)
df = df.replace([-5, -4, -3, -2, -1], np.nan)
df.dropna(axis = 0, inplace = True)

df['logWage'] = df.wage.apply(np.log)
df['white'] = df.ethnicity.apply(lambda x:  1 if x == 1 else 0)
df['black'] = df.ethnicity.apply(lambda x: 1 if x == 2 else 0)
df['hispanic'] = df.ethnicity.apply(lambda x: 1 if x == 3 else 0)
df['smsaDummy'] = df.smsa.apply(lambda x: 1 if x == 0 else 0)

df['expSq'] = df.experience.apply(lambda x: x ** 2)
df['ageSq'] = df.age.apply(lambda x: x ** 2)

df.head(5)

,CASEID,ethnicity,SAMPLE_SEX,age,ASVABa,ASVABb,ASVABc,ASVABd,ASVABe,ASVABf,...,unionCoverage,experience,tenure,logWage,white,black,hispanic,smsaDummy,expSq,ageSq
1,2,1.0,2,22.0,6.0,8.0,15.0,6.0,29.0,52.0,...,0,0.806630,283.0,6.347389,1,0,0,0,0.650652,484.0
3,4,1.0,2,18.0,15.0,18.0,29.0,12.0,46.0,62.0,...,0,0.000000,0.0,5.828946,1,0,0,0,0.000000,324.0
4,5,1.0,1,21.0,25.0,30.0,35.0,13.0,35.0,55.0,...,1,0.850829,338.0,6.109248,1,0,0,0,0.723910,441.0
5,6,1.0,1,20.0,23.0,30.0,35.0,15.0,45.0,68.0,...,1,0.408840,16.0,5.814131,1,0,0,0,0.167150,400.0
7,8,1.0,2,22.0,18.0,13.0,35.0,12.0,24.0,48.0,...,0,0.469613,185.0,6.131226,1,0,0,0,0.220537,484.0


In [7]:
for i in 'abcdefgh':
  model = 'ASVAB' + str(i) + ' ~ age'
  cognitiveModel = smf.ols(model, data = df).fit()
  df['cognitive' + str(i)] = cognitiveModel.resid
  
df['cognitiveAbilities'] = sum([df['cognitive' + str(i)] for i in 'abcdefgh'])
df.drop(labels = ['ASVAB' + str(i) for i in 'abcdefgh'], axis = 1, inplace = True)
df.drop(labels = ['cognitive' + str(i) for i in 'abcdefgh'], axis = 1, inplace = True)
df.head(5)

,CASEID,ethnicity,SAMPLE_SEX,age,marriage,educ,smsa,wage,unionCoverage,experience,tenure,logWage,white,black,hispanic,smsaDummy,expSq,ageSq,cognitiveAbilities
1,2,1.0,2,22.0,1.0,9.0,3.0,571.0,0,0.806630,283.0,6.347389,1,0,0,0,0.650652,484.0,-46.619455
3,4,1.0,2,18.0,2.0,9.0,2.0,340.0,0,0.000000,0.0,5.828946,1,0,0,0,0.000000,324.0,43.261043
4,5,1.0,1,21.0,1.0,15.0,1.0,450.0,1,0.850829,338.0,6.109248,1,0,0,0,0.723910,441.0,59.850669
5,6,1.0,1,20.0,1.0,14.0,2.0,335.0,1,0.408840,16.0,5.814131,1,0,0,0,0.167150,400.0,91.320794
7,8,1.0,2,22.0,2.0,12.0,1.0,460.0,0,0.469613,185.0,6.131226,1,0,0,0,0.220537,484.0,-12.619455


Let's take a look at the dataset now!

In [8]:
df = df[df.wage > 0]
df = df[df.educ < 25]
df = df[df.educ > 0]
df = df[df.age > 0]
df.describe()

,CASEID,ethnicity,SAMPLE_SEX,age,marriage,educ,smsa,wage,unionCoverage,experience,tenure,logWage,white,black,hispanic,smsaDummy,expSq,ageSq,cognitiveAbilities
count,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000,8313.000000
mean,5939.349814,1.320823,1.479249,19.789125,1.235054,11.726212,1.380849,487.833754,0.146999,0.453057,77.720919,6.068140,0.728858,0.221460,0.049681,0.264525,0.291281,396.729099,0.028496
std,3570.688459,0.563290,0.499599,2.262792,0.490122,1.854840,1.080842,275.582396,0.354126,0.293310,114.645558,0.510258,0.444576,0.415255,0.217299,0.441107,0.279478,89.630440,50.432817
min,2.000000,1.000000,1.000000,15.000000,1.000000,1.000000,0.000000,13.000000,0.000000,0.000000,0.000000,2.564949,0.000000,0.000000,0.000000,0.000000,0.000000,225.000000,-182.089580
25%,2903.000000,1.000000,1.000000,18.000000,1.000000,11.000000,0.000000,335.000000,0.000000,0.193370,10.000000,5.814131,0.000000,0.000000,0.000000,0.000000,0.037392,324.000000,-34.738957
50%,5641.000000,1.000000,1.000000,20.000000,1.000000,12.000000,1.000000,407.000000,0.000000,0.447514,26.000000,6.008813,1.000000,0.000000,0.000000,0.000000,0.200269,400.000000,5.380545
75%,8902.000000,2.000000,2.000000,22.000000,1.000000,12.000000,2.000000,563.000000,0.000000,0.712707,101.000000,6.333280,1.000000,0.000000,0.000000,1.000000,0.507952,484.000000,39.201292
max,12686.000000,3.000000,2.000000,24.000000,3.000000,18.000000,3.000000,7500.000000,1.000000,1.000000,1586.000000,8.922658,1.000000,1.000000,1.000000,1.000000,1.000000,576.000000,122.201292


## Building the models

Next two cells do OLS on females and males respectively using only education as a regressor.

In [9]:
dfW = df[df.SAMPLE_SEX == 2]
print(len(dfW))

results = smf.ols('logWage ~ educ', data=dfW).fit()


print(results.summary())

3984
                            OLS Regression Results                            
Dep. Variable:                logWage   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     353.4
Date:                Fri, 23 Mar 2018   Prob (F-statistic):           1.33e-75
Time:                        14:14:45   Log-Likelihood:                -2564.7
No. Observations:                3984   AIC:                             5133.
Df Residuals:                    3982   BIC:                             5146.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.0727      0.049    103.671    

In [10]:
dfM = df[df.SAMPLE_SEX == 1]
print(len(dfM))

results = smf.ols('logWage ~ educ', data=dfM).fit()


print(results.summary())

4329
                            OLS Regression Results                            
Dep. Variable:                logWage   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     301.5
Date:                Fri, 23 Mar 2018   Prob (F-statistic):           2.40e-65
Time:                        14:14:46   Log-Likelihood:                -3196.8
No. Observations:                4329   AIC:                             6398.
Df Residuals:                    4327   BIC:                             6410.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.3256      0.048    111.096    

As we can see, females have stronger effect of education on the wage. Which is usually explained by the stronger selection bias due to discrimination.

Now we include age, squared age, union coverage, marital status, SMSA residency and ethnicity, but no cognitive abilities and experience, for females and males respectively.

In [11]:
results = smf.ols('logWage ~ educ + age + ageSq + unionCoverage + marriage + smsa + black + hispanic', data=dfW).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                logWage   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     67.32
Date:                Fri, 23 Mar 2018   Prob (F-statistic):          4.73e-104
Time:                        14:14:48   Log-Likelihood:                -2481.0
No. Observations:                3984   AIC:                             4980.
Df Residuals:                    3975   BIC:                             5037.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         4.2620      0.601      7.088

In [12]:
results = smf.ols('logWage ~ educ + age + ageSq + unionCoverage + marriage + smsa + black + hispanic', data=dfM).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                logWage   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     121.3
Date:                Fri, 23 Mar 2018   Prob (F-statistic):          7.91e-184
Time:                        14:14:49   Log-Likelihood:                -2903.9
No. Observations:                4329   AIC:                             5826.
Df Residuals:                    4320   BIC:                             5883.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         2.9694      0.583      5.092

Although the regression might be multicolinear at that point and the results don't match exactly with the original paper, we still see the same trends as in 1995 work: effect of the education goes down with the presence of controls, age and union coverage are strong positive signals for higher wage and being a minority will probably affect your wealth outcome not in a good way. Along with the Blackburn paper! Effect of education is still stronger for the females.

Let's add the experience now!

In [13]:
results = smf.ols('logWage ~ educ + experience + expSq + unionCoverage + marriage + smsa + black + hispanic', data=dfW).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                logWage   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     81.42
Date:                Fri, 23 Mar 2018   Prob (F-statistic):          3.85e-125
Time:                        14:14:51   Log-Likelihood:                -2431.8
No. Observations:                3984   AIC:                             4882.
Df Residuals:                    3975   BIC:                             4938.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         5.1217      0.053     96.527

In [14]:
results = smf.ols('logWage ~ educ + experience + expSq + unionCoverage + marriage + smsa + black + hispanic', data=dfM).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                logWage   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     108.5
Date:                Fri, 23 Mar 2018   Prob (F-statistic):          1.38e-165
Time:                        14:14:52   Log-Likelihood:                -2946.3
No. Observations:                4329   AIC:                             5911.
Df Residuals:                    4320   BIC:                             5968.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         5.1754      0.052     99.671

Strong multicollinearity could be the issue that gives the experience coefficient significanlty lower than in the original paper. Notice that effect of the education decreased again as predicted. Another explanation for insignificance of the experience might be the fact that I am using 1979-1981 data where respondents didn't have as much experience and thus the effect of the experience was lower compared to 1980-1985 used in the original study. Also, I don't get rid of self-employed people and agricultural workers, which might change importance of experience too.

In [15]:
results = smf.ols('logWage ~ educ + experience + expSq + unionCoverage + marriage + smsa + black + hispanic + cognitiveAbilities', data=dfW).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                logWage   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     74.63
Date:                Fri, 23 Mar 2018   Prob (F-statistic):          6.08e-128
Time:                        14:14:53   Log-Likelihood:                -2423.0
No. Observations:                3984   AIC:                             4866.
Df Residuals:                    3974   BIC:                             4929.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              5.1936      0

In [16]:
results = smf.ols('logWage ~ educ + experience + expSq + unionCoverage + marriage + smsa + black + hispanic + cognitiveAbilities', data=dfM).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                logWage   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     96.45
Date:                Fri, 23 Mar 2018   Prob (F-statistic):          1.38e-164
Time:                        14:14:53   Log-Likelihood:                -2946.2
No. Observations:                4329   AIC:                             5912.
Df Residuals:                    4319   BIC:                             5976.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              5.1660      0

As concluded by the paper, adding estimates of cognitive abilities lower importance of education (for females). At this point, due to the fact that in my replication I derive different estimate for cognitive abilities the disparity is significantly higher and for the males cognitive abilities are not an important factor at all (maybe because I am not getting rid of self-employed and agricultural workers)

In [17]:
results = smf.ols('logWage ~ educ + tenure + unionCoverage + marriage + smsa + black + hispanic', data=dfM).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                logWage   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                  0.136
Method:                 Least Squares   F-statistic:                     98.49
Date:                Fri, 23 Mar 2018   Prob (F-statistic):          5.85e-134
Time:                        14:14:54   Log-Likelihood:                -3022.2
No. Observations:                4329   AIC:                             6060.
Df Residuals:                    4321   BIC:                             6111.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         5.1038      0.052     98.286

Replacing experience with tenure does not change much.

Overidentification test and Hausman test.

Asymptotically Hausman test converges to F-test which means we can just look at the F-value, which leaves us with the result that the paper was correct saying that regression of wage on education by itself would result in overstated effect of the education.
The fact that condition number is large says that there might be an overidentification problem.

## Conclusion
It is a great paper. Main results were replicated, whilst some more complicated techniques and data selection process were simplified, which still gives the results stated in the article. As a personal result, I got to investigate the NSLY dataset structure and statistical models in Python package.